Importamos las librerias a usar

In [1]:
import pandas as pd
from textblob import TextBlob
import spacy
import numpy as np

# Creamos un objeto con la ruta del archivo
archivo = r'Datasets/csv/user_reviews_normalizado.csv'


In [14]:
df = pd.read_csv(archivo)
df2 = df.copy()

In [15]:
# Normalizar valores
df2['review'] = df2['review'].replace(['10/10', '100/100'], 'Good', regex=True)

# Transformar todos los datos a tipo string
df2['review'] = df2['review'].astype(str)

In [16]:
# Cargar el modelo de lenguaje
nlp = spacy.load('en_core_web_sm')

# Definimos función para procesar el texto
def process_text_spacy(text):
    doc = nlp(text)
    tokens = [token.text for token in doc]
    lemmatized_words = [token.lemma_ for token in doc]
    return {
        'tokens': tokens,
        'lemmatized': ' '.join(lemmatized_words)
    }

# Aplicar la función a la columna 'review' del DataFrame
df2[['tokens', 'lemmatized']] = df2['review'].apply(lambda x: pd.Series(process_text_spacy(x)))

In [17]:
# Definimos la función de análisis de sentimiento
def analizar_sentimiento(text, recommend):
    if pd.isnull(text):
        return 1  # Neutral si el texto es nulo

    # Obtener la polaridad del análisis de sentimiento usando TextBlob
    blob = TextBlob(text)
    polaridad = blob.sentiment.polarity

    # Ajustar la polaridad según el valor de 'recommend'
    if recommend:
        polaridad += 0.5
    else:
        polaridad -= 0.5

    # Clasificar el sentimiento según la polaridad ajustada
    if polaridad <= -0.05:
        return 0  # Malo
    elif polaridad >= 0.05:
        return 2  # Bien
    else:
        return 1  # Neutral

# Crear la columna 'analisis_sentimiento' en el DataFrame usando la nueva función
df2['analisis_sentimiento'] = df2.apply(
    lambda row: analizar_sentimiento(row['lemmatized'], row['recommend']), axis=1
)

# Eliminamos la columna 'lemmatized' y "tokens" que ya no son necesarias
columnas= ["lemmatized", "tokens"]
df2.drop(columns=columnas, inplace=True)

In [18]:
# Reseñas con sentimiento positivo pero con una recomendación negativa.
positivo_negativo = df2[(df2['analisis_sentimiento'] == 2) & (df2['recommend'] == False)]

# Reseñas con sentimiento negativo pero con una recomendación positiva.
negativo_positivo = df2[(df2['analisis_sentimiento'] == 0) & (df2['recommend'] == True)]

# Conteo de reseñas con sentimiento positivo y recomendación negativa.
total_positivos_negativos = len(positivo_negativo)

# Conteo de reseñas con sentimiento negativo y recomendación positiva.
total_negativos_positivos = len(negativo_positivo)

print("Conteo de reseñas con sentimiento negativo y recomendación positiva:", total_negativos_positivos)
print("Conteo de reseñas con sentimiento positivo y recomendación negativa:", total_positivos_negativos)

Conteo de reseñas con sentimiento negativo y recomendación positiva: 380
Conteo de reseñas con sentimiento positivo y recomendación negativa: 905


In [19]:
total_errores = 380 + 905
total_filas = 59333
porcentaje_error = (total_errores / total_filas) * 100

print("Porcentaje de errores:", porcentaje_error)

Porcentaje de errores: 2.1657425041713716


In [20]:
df2.to_csv("user_reviews_Analisis_sentimientos.csv", index=False)